In [1]:
import sys
sys.path.append("../")
sys.path.append("../../")

In [2]:
import numpy as np
from Onsager_calc_db import *
import onsager.crystal as crystal
from states import *
from stars import *
from vector_stars import *
import time

/usr/local/lib/python3.6/dist-packages/onsager-1.3.3-py3.6.egg/onsager/cluster.py:835: NumbaDeprecationWarning: The 'numba.jitclass' decorator has moved to 'numba.experimental.jitclass' to better reflect the experimental nature of the functionality. Please update your imports to accommodate this change and see http://numba.pydata.org/numba-doc/latest/reference/deprecation.html#change-of-jitclass-location for the time frame.


In [3]:
# make a BCC lattice
# We'll modify the jumpnetwork to keep only the 60 degree reorientational jumps.
latt = np.array([[1., 0., 0.], [0., 1., 0.], [0., 0., 1.]]) * 0.28553
Fe = crystal.Crystal(latt, [[np.array([0., 0., 0.]), np.array([0.5, 0.5, 0.5])]], ["Fe"])
# Now give it the orientations - for BCC it's [110]
o = np.array([1.,1.,0.])/np.linalg.norm(np.array([1.,1.,0.]))*0.126*2
famp0 = [o.copy()]
family = [famp0]
pdbcontainer_fe = dbStates(Fe, 0, family)
mdbcontainer_fe = mStates(Fe, 0, family)
jset0, jset2 = pdbcontainer_fe.jumpnetwork(0.27, 0.01, 0.01), mdbcontainer_fe.jumpnetwork(0.27, 0.01, 0.01)
print(Fe)

#Lattice:
  a1 = [0.142765 0.142765 0.142765]
  a2 = [-0.142765  0.142765 -0.142765]
  a3 = [-0.142765 -0.142765  0.142765]
#Basis:
  (Fe) 0.0 = [0. 0. 0.]


In [4]:
# Modify jnet0
jnet0 = jset0[0]
jnet0_indexed = jset0[1]
# Let's try to sort the jumps according to closest distance
# except rotational jumps, we don't want them.
z = np.zeros(3)
indices = []

for jt, jlist in enumerate(jnet0):
    if np.allclose(jnet0_indexed[jt][0][1], z):
        continue
    indices.append(jt)
    
def sortkey(entry):
    jmp = jnet0[entry][0]
    or1 = pdbcontainer_fe.iorlist[jmp.state1.iorind][1]
    or2 = pdbcontainer_fe.iorlist[jmp.state2.iorind][1]
    dx = disp(pdbcontainer_fe, jmp.state1, jmp.state2)
    dx1 = np.linalg.norm(jmp.c1*or1/2.)
    dx2 = np.linalg.norm(dx + jmp.c2*or2/2. - jmp.c1*or1/2.)
    dx3 = np.linalg.norm(-jmp.c2*or2/2.)
    return dx1+dx2+dx3
ind_sort = sorted(indices, key=sortkey)

In [5]:
# Let's check if we got the correct jump
print(jnet0[ind_sort[0]][0])

Jump object:
Initial state:
	dumbbell : (i, or) index = 3, lattice vector = [0 0 0]
Final state:
	dumbbell : (i, or) index = 5, lattice vector = [1 1 1]
Jumping from c1 = 1 to c2 = 1



In [6]:
pdbcontainer_fe.iorlist

[(0, array([ 1.78190909e-01,  1.78190909e-01, -5.24417134e-18])),
 (0, array([ 0.        , -0.17819091,  0.17819091])),
 (0, array([-5.24417134e-18,  1.78190909e-01,  1.78190909e-01])),
 (0, array([-1.78190909e-01, -5.24417134e-18,  1.78190909e-01])),
 (0, array([0.17819091, 0.        , 0.17819091])),
 (0, array([ 0.17819091, -0.17819091,  0.        ]))]

In [7]:
# take only the lowest displacement jump
# that is the jump we want.
jset0new = ([jnet0[ind_sort[0]]], [jnet0_indexed[ind_sort[0]]])

In [8]:
# Now, we modify the mixed dumbbell jumpnetwork to also give the lowest displacement jump
# Modify jnet0
jnet2 = jset2[0]
jnet2_indexed = jset2[1]
# Let's try to sort the jumps according to closest distance
# we don't want the rotational jumps as before.
z = np.zeros(3)
indices2 = []
for jt, jlist in enumerate(jnet2):
    if np.allclose(jnet2_indexed[jt][0][1], z):
        continue
#     # get the 90 degree reorientations
#     or1 = mdbcontainer_fe.iorlist[jlist[0].state1.db.iorind][1]
#     or2 = mdbcontainer_fe.iorlist[jlist[0].state2.db.iorind][1]
#     if not np.dot(or1, or2) <= 1e-8:
#         continue
    indices2.append(jt)    
print(indices2)

def sortkey2(entry):
    jmp = jnet2[entry][0]
    or1 = mdbcontainer_fe.iorlist[jmp.state1.db.iorind][1]
    or2 = mdbcontainer_fe.iorlist[jmp.state2.db.iorind][1]
    dx = disp(mdbcontainer_fe, jmp.state1, jmp.state2)
    # c1 and c2 are always +1 for mixed dumbbell jumps.
    dx1 = np.linalg.norm(jmp.c1*or1/2.)
    dx2 = np.linalg.norm(dx + jmp.c2*or2/2. - jmp.c1*or1/2.)
    dx3 = np.linalg.norm(-jmp.c2*or2/2.)
    return dx1+dx2+dx3
ind_sort2 = sorted(indices2, key=sortkey2)
print(ind_sort2)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16]
[16, 15, 3, 10, 4, 0, 11, 2, 12, 1, 6, 5, 8, 9, 7, 14, 13]


In [9]:
# check if we have the correct type of jump
print(jnet2[ind_sort2[0]][0])

Jump object:
Initial state:
	Solute loctation:basis index = 0, lattice vector = [0 0 0]
	dumbbell : (i, or) index = 4, lattice vector = [0 0 0]
Final state:
	Solute loctation :basis index = 0, lattice vector = [1 1 1]
	dumbbell : (i, or) index = 8, lattice vector = [1 1 1]
Jumping from c1 = 1 to c2 = 1


In [10]:
for tup in mdbcontainer_fe.iorlist:
    print(tup)

(0, array([ 1.78190909e-01,  1.78190909e-01, -5.24417134e-18]))
(0, array([ 0.        , -0.17819091,  0.17819091]))
(0, array([ 0.        ,  0.17819091, -0.17819091]))
(0, array([-5.24417134e-18,  1.78190909e-01,  1.78190909e-01]))
(0, array([-1.78190909e-01, -5.24417134e-18,  1.78190909e-01]))
(0, array([-1.78190909e-01, -1.78190909e-01,  5.24417134e-18]))
(0, array([0.17819091, 0.        , 0.17819091]))
(0, array([-0.17819091,  0.        , -0.17819091]))
(0, array([ 5.24417134e-18, -1.78190909e-01, -1.78190909e-01]))
(0, array([ 1.78190909e-01,  5.24417134e-18, -1.78190909e-01]))
(0, array([ 0.17819091, -0.17819091,  0.        ]))
(0, array([-0.17819091,  0.17819091,  0.        ]))


In [11]:
# take only the lowest displacement jump
# that is the jump we want.
jset2new = ([jnet2[ind_sort2[0]]], [jnet2_indexed[ind_sort2[0]]])

In [12]:
# Now, we construct the Onsager calculator with these non-local jump sets.
start = time.time()
onsagercalculator = dumbbellMediated(pdbcontainer_fe, mdbcontainer_fe, jset0new, jset2new, 0.27,
                                     0.01, 0.01, 0.01, NGFmax=4, Nthermo=1)
print("onsager calculator initiation time = {}".format(time.time() - start))

initializing thermo
initializing kin
initializing NN
built shell 1: time - 0.05468630790710449
built shell 2: time - 0.948483943939209
grouped states by symmetry: 0.9861617088317871
built mixed dumbbell stars: 0.00046515464782714844
built jtags2: 0.0005040168762207031
built mixed indexed star: 0.009659767150878906
building star2symlist : 0.00018477439880371094
building bare, mixed index dicts : 0.00037026405334472656
2NN Shell initialization time: 2.903938055038452

generating thermodynamic shell
built shell 1: time - 0.02448439598083496
grouped states by symmetry: 0.09008097648620605
built mixed dumbbell stars: 0.0005519390106201172
built jtags2: 0.00023508071899414062
built mixed indexed star: 0.0075321197509765625
building star2symlist : 4.267692565917969e-05
building bare, mixed index dicts : 0.00011992454528808594
thermodynamic shell generated: 0.20065665245056152
Total number of states in Thermodynamic Shell - 54, 12
generating kinetic shell
built shell 1: time - 0.03433632850646

In [13]:
onsagercalculator.om1types

[0, 0, 0, 0, 0, 0, 0, 0]

In [14]:
# First, we must also modify the omega3 and omega4 jump lists
# NOTE - I am modifying these lists in place right now.
# But I must come up with a way to do this separately
len(onsagercalculator.jnet43)

26

In [15]:
jnet43 = onsagercalculator.jnet43
jnet43_indexed = onsagercalculator.jnet43_indexed
# Let's try to sort the jumps according to closest distance
# we don't want the rotational jumps as before.

z = np.zeros(3)
indices43 = []
for jt, jlist in enumerate(jnet43):
    if np.allclose(jnet43_indexed[jt][0][1], z):
        continue
    indices43.append(jt)    
# print(indices43)

def sortkey43(entry):
    jmp = jnet43[entry][0] # This is an omega4 jump
    if not jmp.c2 == -1:
        print(c2)
    or1 = pdbcontainer_fe.iorlist[jmp.state1.db.iorind][1]
    or2 = mdbcontainer_fe.iorlist[jmp.state2.db.iorind][1]
    dx = disp4(pdbcontainer_fe, mdbcontainer_fe, jmp.state1, jmp.state2)
    # remember that c2 is -1 for an omega4 jump
    dx1 = np.linalg.norm(jmp.c1*or1/2.)
    dx2 = np.linalg.norm(dx - or2/2. - jmp.c1*or1/2.)
    dx3 = np.linalg.norm(jmp.c2*or2/2.)
    return dx1+dx2+dx3

ind_sort43 = sorted(indices43, key=sortkey43)
print(ind_sort43)

[10, 8, 4, 23, 12, 24, 2, 22, 0, 9, 14, 6, 11, 17, 18, 19, 16, 15, 3, 20, 13, 25, 5, 21, 7, 1]


In [16]:
# check if we have the correct jump
print(jnet43[ind_sort43[0]][0])

Jump object:
Initial state:
	Solute loctation:basis index = 0, lattice vector = [0 0 0]
	dumbbell : (i, or) index = 0, lattice vector = [ 0  0 -1]
Final state:
	Solute loctation :basis index = 0, lattice vector = [0 0 0]
	dumbbell : (i, or) index = 1, lattice vector = [0 0 0]
Jumping from c1 = -1 to c2 = -1


In [17]:
pdbcontainer_fe.iorlist

[(0, array([ 1.78190909e-01,  1.78190909e-01, -5.24417134e-18])),
 (0, array([ 0.        , -0.17819091,  0.17819091])),
 (0, array([-5.24417134e-18,  1.78190909e-01,  1.78190909e-01])),
 (0, array([-1.78190909e-01, -5.24417134e-18,  1.78190909e-01])),
 (0, array([0.17819091, 0.        , 0.17819091])),
 (0, array([ 0.17819091, -0.17819091,  0.        ]))]

In [18]:
mdbcontainer_fe.iorlist

[(0, array([ 1.78190909e-01,  1.78190909e-01, -5.24417134e-18])),
 (0, array([ 0.        , -0.17819091,  0.17819091])),
 (0, array([ 0.        ,  0.17819091, -0.17819091])),
 (0, array([-5.24417134e-18,  1.78190909e-01,  1.78190909e-01])),
 (0, array([-1.78190909e-01, -5.24417134e-18,  1.78190909e-01])),
 (0, array([-1.78190909e-01, -1.78190909e-01,  5.24417134e-18])),
 (0, array([0.17819091, 0.        , 0.17819091])),
 (0, array([-0.17819091,  0.        , -0.17819091])),
 (0, array([ 5.24417134e-18, -1.78190909e-01, -1.78190909e-01])),
 (0, array([ 1.78190909e-01,  5.24417134e-18, -1.78190909e-01])),
 (0, array([ 0.17819091, -0.17819091,  0.        ])),
 (0, array([-0.17819091,  0.17819091,  0.        ]))]

In [19]:
onsagercalculator.regenerate43([ind_sort43[0]])

In [20]:
# 1.  First get the rates and thermodynamic data
# All the energies of the "mixed" and pure dumbbells will be the same,
# All the jump rates will be the same
    # Since we have only one type each of omega0, omega2 and omega43 jumps, set their rates to zero.
    # All omega1 rates will be the same as the above rate.
# The "solute" energies will be zero since we are dealing with a chemically identical tracer.
# All interaction energies will be zero.

# 1a. Energies and pre-factors
kT = 1

predb0, enedb0 = np.ones(len(onsagercalculator.vkinetic.starset.pdbcontainer.symorlist)), \
                 np.random.rand(len(onsagercalculator.vkinetic.starset.pdbcontainer.symorlist))

preS, eneS = np.ones(
    len(onsagercalculator.vkinetic.starset.crys.sitelist(onsagercalculator.vkinetic.starset.chem))), \
             np.zeros(len(onsagercalculator.vkinetic.starset.crys.sitelist(
                 onsagercalculator.vkinetic.starset.chem)))

# These are the interaction or the excess energies and pre-factors for solutes and dumbbells.
# The energies will all be zero.
preSdb, eneSdb = np.ones(onsagercalculator.thermo.mixedstartindex), \
                 np.zeros(onsagercalculator.thermo.mixedstartindex)

predb2, enedb2 = predb0.copy(), enedb0.copy()

preT0, eneT0 = np.ones(len(onsagercalculator.vkinetic.starset.jnet0)), np.random.rand(
    len(onsagercalculator.jnet0))
preT2, eneT2 = preT0.copy(), eneT0.copy()
preT1, eneT1 = np.ones(len(onsagercalculator.jnet1)), np.array([eneT0[onsagercalculator.om1types[jt]] for jt in
                                                                range(len(onsagercalculator.jnet1))])

preT43, eneT43 = np.ones(len(onsagercalculator.jnet43)), eneT0.copy()

In [21]:
# 1b. Now get the beta*free energy values.
bFdb0, bFdb2, bFS, bFSdb, bFT0, bFT1, bFT2, bFT3, bFT4 = \
    onsagercalculator.preene2betafree(kT, predb0, enedb0, preS, eneS, preSdb, eneSdb, predb2, enedb2,
                                           preT0, eneT0, preT2, eneT2, preT1, eneT1, preT43, eneT43)

In [22]:
len(onsagercalculator.jnet1)

8

In [23]:
# get the probabilities and other data from L_ij
L0bb,(L_uc_aa,L_c_aa), (L_uc_bb,L_c_bb), (L_uc_ab,L_c_ab), GF_total, GF20, betaFs, del_om, part_func, probs,\
omegas, stateprobs =\
onsagercalculator.L_ij(bFdb0, bFT0, bFdb2, bFT2, bFS, bFSdb, bFT1, bFT3, bFT4)

In [25]:
L_aa = L_uc_aa + L_c_aa

In [26]:
L_ab = L_uc_ab + L_c_ab

In [27]:
L_aa[0][0]/L_ab[0][0]

0.4126434097962868